In [1]:
from tensorflow.examples.tutorials.mnis
t import input_data
mnist = input_data.read_data_sets("../../MNIST_data/", one_hot=True)

W0225 14:50:27.328722 4579485120 deprecation.py:323] From <ipython-input-1-3c4f46c1a93b>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0225 14:50:27.329906 4579485120 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0225 14:50:27.330456 4579485120 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will 

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz


W0225 14:50:27.554586 4579485120 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import os

from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Reshape, Embedding, Concatenate
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, Flatten
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.layers.merge import concatenate
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils, plot_model
from keras.datasets import mnist

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image

import numpy as np
np.random.seed(1)

import random
import math

Using TensorFlow backend.


In [4]:
(X_train, y_train), (_, _) = mnist.load_data()

X_train = X_train.reshape(-1, 28*28)/255.

# y_train = y_train.reshape(-1, 1)
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_train = y_train.reshape(-1, 10)

(60000, 10)


In [5]:
x_dim = X_train.shape[1] # 784
y_class = 10
z_dim = 100
c_dim = 10
h_dim = 128
batch_size = 128

x = Input(shape=(x_dim,))
c = Input(shape=(c_dim,))
D_h = Concatenate(axis=1)([x, c])
D_h = Dense(h_dim)(D_h)
D_h = LeakyReLU(0.2)(D_h)
D_o = Dense(1, activation='sigmoid')(D_h)
DM = Model([x, c], D_o)

z = Input(shape=(z_dim,))
c = Input(shape=(c_dim,))
G_h = Concatenate(axis=1)([z, c])
G_h = Dense(h_dim)(G_h)
G_h = LeakyReLU(0.2)(G_h)
G_o = Dense(x_dim, activation='sigmoid')(G_h)
GM = Model([z, c], G_o)

# del DM
# DM = load_model('DM.h5')
# del GM
# GM = load_model('GM.h5')

DM.trainable = True
DM.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

g = GM([z, c])
DM.trainable = False
y = DM([g, c])
AM = Model([z, c], y)
AM.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

GM.summary()
DM.summary()
AM.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 110)          0           input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 128)          14208       concatenate_4[0][0]              
__________

/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [7]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m,n])

def sample_C(m):
#     return np.random.randint(0, 10, m).reshape(-1, 1)
    return np.random.multinomial(1, 10*[0.1], size=m)

if not os.path.exists('out/'):
    os.makedirs('out/')

real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))
    
i=0
for it in range(1000000):
    r = np.random.randint(0, X_train.shape[0], batch_size)
    x_real, y_real = X_train[r], y_train[r]
    
    z_samples = sample_Z(batch_size, z_dim)
    x_fake = GM.predict([z_samples, y_real])
    
    D_loss_real, D_acc_real = DM.train_on_batch([x_real, y_real], real)
    D_loss_fake, D_acc_fake = DM.train_on_batch([x_fake, y_real], fake)
    
    D_loss = 0.5*(D_loss_real + D_loss_fake)
    D_acc = 0.5*(D_acc_real + D_acc_fake)
    
    z_samples = sample_Z(batch_size, z_dim)
    y_samples = sample_C(batch_size)
    
    G_loss, G_acc = AM.train_on_batch([z_samples, y_samples], real)
    
    if it%10000==0:
        DM.save('DM.h5')
        GM.save('GM.h5')
        
        for j in range(y_class):
            y_samples = np.ones((16,1))*j
            y_samples = np_utils.to_categorical(y_samples, num_classes=c_dim)
            samples = GM.predict([sample_Z(16, z_dim), y_samples])
            fig = plot(samples)
            plt.savefig('out/{}.png'.format(str(i).zfill(3) + '_' + str(j)), bbox_inches='tight')
            plt.close(fig)
        i+=1
        
        print('iter:{}'.format(it))
        print('[D loss:{:.4}, acc:{:.4}] [G loss:{:.4}, acc:{:.4}]'.format(D_loss, D_acc, G_loss, G_acc))
        print()

iter:0
[D loss:0.3621, acc:1.0] [G loss:2.289, acc:0.0]

iter:10000
[D loss:0.2516, acc:0.8867] [G loss:2.637, acc:0.1094]

iter:20000
[D loss:0.4532, acc:0.7695] [G loss:1.844, acc:0.125]



KeyboardInterrupt: 